In [1]:
from service.MatchSys.match_sys import MatchSys
from service import config
config.initialize()
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\model.pkl'
    )

1714106437376024576
Loading weights from local directory
# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.


In [2]:
from service.MatchSys.trainers import Trainer
from service.MatchSys.utils import print_progress_bar


class QATrainer(Trainer):
    """
    Allows a chat bot to be trained using a list of strings
    where the list represents a conversation.
    """

    def train(self, conversation, **kwargs):
        """
        {Q:[A1,A2...]}
        Train the chat bot based on the provided list of
        statements that represents a single conversation.
        """
        source = kwargs.get('source', 'TRAIN_DATA')
        conversation_text = kwargs.get('conversation', 'TRAIN_DATA')
        statements_to_create = []
        previous_id = 0
        next_id = 0
        input_statements = self.chatbot.message_adapter.process_list(list(conversation.keys()),**kwargs)
        for index,input_statement in enumerate(input_statements):
                if self.show_training_progress:
                    print_progress_bar(
                        'QA Trainer',
                        index + 1, len(conversation)
                    )
                try:
                    statements_to_create.append(input_statement)
                    kwargs['id'] = next_id
                    kwargs['type_of'] = 'A'
                    kwargs['persona'] = 'bot:'+self.chatbot.name
                    for statement in  self.chatbot.message_adapter.process_list(conversation[input_statement.text],**kwargs):

                        statements_to_create.append(statement)
                except Exception as e:
                    print(statement)
        self.chatbot.storage.create_many(statements_to_create)
        self.chatbot.docvector_tool.train(statements_to_create)

In [3]:
import pandas as pd
map = {}
# data = pd.read_excel(r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx',header=None, sheet_name=0)
data = pd.read_excel(r"D:\temp\Downloads\傲娇系二次元bot词库5千词V1.2 - Test.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]

In [4]:
trainer = QATrainer(ms)

In [5]:
trainer.train(map)

QA Trainer: [####################] 100%


MemoryError: 

In [ ]:
res = ms.get_response('你')

In [10]:
a = []
for i in range(ms.message_adapter.snowflake.get_id()):
    a.append(i)


KeyboardInterrupt: 

In [ ]:
kwargs = {}
conversation_text = kwargs.get('conversation', 'TRAIN_DATA')
statements_to_create = []
previous_id = 0
next_id = 0
kwargs['type_of'] = 'Q'
kwargs['persona'] = 'user:*'
input_statements = ms.message_adapter.process_list(list(map.keys()),**kwargs)
for index,input_statement in enumerate(input_statements):
    statements_to_create.append(input_statement)
    kwargs['id'] = next_id
    kwargs['type_of'] = 'A'
    kwargs['persona'] = 'bot:'+ms.name

data = ms.docvector_tool.build_tokenzied(statements_to_create)

In [ ]:
temp = []
count = 0
for tag in data:
    tag.tags[0] = count
    temp.append(tag)
    count = count + 1

In [ ]:
from gensim.models import Doc2Vec
ms.docvector_tool.model = Doc2Vec(temp,dm=1, window=8, min_count=5, workers=4)
ms.docvector_tool.model.train(temp,total_examples=ms.docvector_tool.model.corpus_count,epochs=10)

In [ ]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import jieba
# 加载数据
documents = []
# 使用count当做每个句子的“标签”，标签和每个句子是一一对应的
count = 0
for line in list(map.keys()):
    documents.append(TaggedDocument(jieba.lcut_for_search(line), [str(line),count]))
    count = count + 1
# 模型训练
model = Doc2Vec(documents, dm=1, window=8, min_count=5, workers=4)
model.train(documents,total_examples=model.corpus_count,epochs=10)


In [ ]:
inferred_vector = model.infer_vector(doc_words=['热', '死', '我', '了'])
sims = model.dv.most_similar([inferred_vector],topn=10)

In [ ]:
for sim in sims:
    print(sim)